In [2]:
import tkinter as tk
from tkinter import filedialog, ttk, simpledialog, messagebox
from PIL import Image, ImageTk
import numpy as np
import cv2

class ImageProcessingApp:
    def __init__(self, master):
        self.master = master
        master.title("Image Processing App")
        master.configure(bg="#D6EAF8")  # Set background color

        self.style = ttk.Style()
        self.style.configure('TFrame', background='#D6EAF8')
        self.style.configure('TLabel', background='#D6EAF8', font=('Arial', 12))
        self.style.configure('TButton', font=('Arial', 10, 'bold'), padding=5)
        self.style.configure('TScale', background='#D6EAF8')

        self.original_img = None
        self.processed_img = None
        self.threshold_value = tk.IntVar()
        self.threshold_value.set(128)

        # Create the status bar
        self.status_bar = ttk.Label(master, text="Ready", relief=tk.SUNKEN, anchor=tk.W, background='#AED6F1', foreground='#1B4F72')
        self.status_bar.pack(side=tk.BOTTOM, fill=tk.X)

        # Create the menu bar
        self.menu_bar = tk.Menu(master)
        master.config(menu=self.menu_bar)

        # Create File menu
        self.file_menu = tk.Menu(self.menu_bar, tearoff=0)
        self.file_menu.add_command(label="Open", command=self.load_image)
        self.file_menu.add_separator()
        self.file_menu.add_command(label="Exit", command=master.quit)
        self.menu_bar.add_cascade(label="File", menu=self.file_menu)

        # Create Edit menu
        self.edit_menu = tk.Menu(self.menu_bar, tearoff=0)
        self.edit_menu.add_command(label="Segment Threshold", command=self.segment_threshold)
        self.edit_menu.add_command(label="Segment Edges", command=self.segment_edges)
        self.edit_menu.add_command(label="Histogram Equalization", command=self.histogram_equalization)
        self.edit_menu.add_command(label="Power-Law Transformation", command=self.get_k_and_transform)
        self.menu_bar.add_cascade(label="Edit", menu=self.edit_menu)

        # Create GUI elements
        self.create_widgets()

    def create_widgets(self):
        # Main frame
        main_frame = ttk.Frame(self.master)
        main_frame.pack(padx=20, pady=20)

        # Original image label
        self.original_label = ttk.Label(main_frame)
        self.original_label.pack(side=tk.LEFT, padx=10, pady=10)

        # Processed image label
        self.processed_label = ttk.Label(main_frame)
        self.processed_label.pack(side=tk.LEFT, padx=10, pady=10)

        # Control frame
        control_frame = ttk.Frame(main_frame)
        control_frame.pack(side=tk.LEFT, padx=10, pady=10)

        # Load image button
        load_button = ttk.Button(control_frame, text="Load Image", command=self.load_image)
        load_button.pack(fill=tk.X, padx=10, pady=5)

        # Threshold slider
        threshold_slider = ttk.Scale(control_frame, from_=0, to=255, orient=tk.HORIZONTAL, variable=self.threshold_value, command=self.segment_threshold)
        threshold_slider_label = ttk.Label(control_frame, text="Threshold Value")
        threshold_slider_label.pack(fill=tk.X, padx=10, pady=5)
        threshold_slider.pack(fill=tk.X, padx=10, pady=5)

        # Segment Edges button
        segment_edges_button = ttk.Button(control_frame, text="Segment Edges", command=self.segment_edges)
        segment_edges_button.pack(fill=tk.X, padx=10, pady=5)

        # Histogram Equalization button
        histogram_button = ttk.Button(control_frame, text="Histogram Equalization", command=self.histogram_equalization)
        histogram_button.pack(fill=tk.X, padx=10, pady=5)

        # Power-Law Transformation button
        power_law_button = ttk.Button(control_frame, text="Power-Law Transformation", command=self.get_k_and_transform)
        power_law_button.pack(fill=tk.X, padx=10, pady=5)

    def load_image(self):
        file_path = filedialog.askopenfilename()
        if file_path:
            try:
                self.original_img = Image.open(file_path)
                self.processed_img = self.original_img.copy()  # Make a copy of original image for processing
                self.display_images()
                self.update_status("Image loaded successfully")
            except Exception as e:
                messagebox.showerror("Error", "Failed to load image.")
                self.update_status("Error loading image")

    def update_status(self, message):
        self.status_bar.config(text=message)

    def display_images(self):
        if self.original_img is not None and self.processed_img is not None:
            original_img_resized = self.original_img.resize((400, 400), Image.LANCZOS)
            processed_img_resized = self.processed_img.resize((400, 400), Image.LANCZOS)

            original_img_tk = ImageTk.PhotoImage(original_img_resized)
            processed_img_tk = ImageTk.PhotoImage(processed_img_resized)

            self.original_label.configure(image=original_img_tk)
            self.original_label.image = original_img_tk

            self.processed_label.configure(image=processed_img_tk)
            self.processed_label.image = processed_img_tk

    def segment_threshold(self, event=None):
        if self.original_img is None:
            return
        threshold_value = self.threshold_value.get()
        self.processed_img = self.original_img.convert('L').point(lambda x: 0 if x < threshold_value else 255, '1')
        self.display_images()

    def segment_edges(self):
        if self.original_img is None:
            return
        cv_image = cv2.cvtColor(np.array(self.original_img), cv2.COLOR_RGB2BGR)
        gray_image = cv2.cvtColor(cv_image, cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(gray_image, 100, 200)  # Apply Canny edge detection
        # Dilate the edges to make them more prominent
        kernel = np.ones((5, 5), np.uint8)
        dilated_edges = cv2.dilate(edges, kernel, iterations=1)
        self.processed_img = Image.fromarray(dilated_edges)
        self.display_images()

    def histogram_equalization(self):
        if self.original_img is None:
            return
        cv_image = cv2.cvtColor(np.array(self.original_img), cv2.COLOR_RGB2BGR)
        equalized_image = cv2.equalizeHist(cv2.cvtColor(cv_image, cv2.COLOR_BGR2GRAY))
        self.processed_img = Image.fromarray(cv2.cvtColor(equalized_image, cv2.COLOR_BGR2RGB))
        self.display_images()

    def power_law_transformation(self, k):
        if self.original_img is None:
            return
        cv_image = cv2.cvtColor(np.array(self.original_img), cv2.COLOR_RGB2BGR)
        normalized_image = cv_image / 255.0
        transformed_image = np.power(normalized_image, k)
        transformed_image = np.uint8(transformed_image * 255)
        self.processed_img = Image.fromarray(cv2.cvtColor(transformed_image, cv2.COLOR_BGR2RGB))
        self.display_images()

    def get_k_and_transform(self):
        k = simpledialog.askfloat("Input", "Enter the value of k for power-law transformation:")
        if k is not None:
            self.power_law_transformation(k)

root = tk.Tk()
app = ImageProcessingApp(root)
root.mainloop()


C:\Users\asus\AppData\Local\Temp\ipykernel_30104\2455893212.py:106: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  original_img_resized = self.original_img.resize((400, 400), Image.LANCZOS)
C:\Users\asus\AppData\Local\Temp\ipykernel_30104\2455893212.py:107: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  processed_img_resized = self.processed_img.resize((400, 400), Image.LANCZOS)
